## Setup

To complete the following guide you will need to install the following packages:
- fireworks-ai
- pandas
- requests

You will also need:

- Fireworks account (https://fireworks.ai/)
- Fireworks API key
- The firectl command-line interface (https://docs.fireworks.ai/tools-sdks/firectl/firectl)

In [2]:
!pip install pandas requests fireworks-ai

In [7]:
import json
import os
import time

from fireworks.client import Fireworks
import pandas as pd
import requests

In [ ]:
# Sign-in to your Fireworks account
!firectl signin

In [8]:
# Make sure you have the FIREWORKS_API_KEY environment variable set to your account's key!
# os.environ['FIREWORKS_API_KEY'] = 'XXX'

client = Fireworks()

# Replace the line below with your Fireworks account id
account_id = 'XXX'

## Problem Definition: Insurance Support Ticket Classifier

*Note: The problem definition, data, and labels used in this example were synthetically generated by Claude 3 Opus*

In the insurance industry, customer support plays a crucial role in ensuring client satisfaction and retention. Insurance companies receive a high volume of support tickets daily, covering a wide range of topics such as billing, policy administration, claims assistance, and more. Manually categorizing these tickets can be time-consuming and inefficient, leading to longer response times and potentially impacting customer experience.

### Task
In last week's exercise, we performed prompt engineering to increase the accuracy of our predictions on the test.tsv dataset to 63.24%.

This week, we will now fine-tune our first model to increase the accuracy even higher!

#### Labeled Data

The data can be found in the week-2 `data` folder.

We will use the following datasets:
- `./data/train.tsv`
- `./data/test.tsv`

In [67]:
training_examples = pd.read_csv('data/train.tsv', sep='\t')
test_examples = pd.read_csv('data/test.tsv', sep='\t')

# In order to not leak information about the test labels into our prompts, the list of possible categories will be defined 
# based on the training labels.
categories = sorted(training_examples['label'].unique().tolist())
categories_str = '\n'.join(categories)

training_tickets = training_examples['text'].tolist()
training_labels = training_examples['label'].tolist()

test_tickets = test_examples['text'].tolist()
test_labels = test_examples['label'].tolist()

### Dataset Curation

We first must transform our dataset into the format expected by Fireworks, and then upload the dataset. The dataset must conform to the schema expected by the Chat Completions API.

See https://docs.fireworks.ai/fine-tuning/fine-tuning-models#conversation for more details

In [68]:
def create_prompt(ticket):
    return f"""classify a customer support ticket into one of the following categories:
<categories>
{categories_str}
</categories>

Here is the customer support ticket:    
<ticket>{ticket}</ticket>

Respond using this format:
<category>The category label you chose goes here</category>"""    

In [69]:
# Converts the training examples to the format expected by Fireworks.
def training_examples_to_json(examples):
    json_objs = list()
    for idx, example in examples.iterrows():  
        user_msg = create_prompt(example['text'])
        asst_msg = f"<category>{example['label']}</category>"
        msg = {"messages": [
            {"role": "user", "content": user_msg}, 
            {"role": "assistant", "content": asst_msg}
        ]}
        json_objs.append(msg)
    
    return json_objs

training_json = training_examples_to_json(training_examples)

In [71]:
# Writes the data to a file so that it can be uploaded to Fireworks
dataset_file_name = 'ticket-classification_training_data.jsonl'
dataset_id = 'ticket-classification-v1'

with open(dataset_file_name, 'w') as f:
    for obj in training_json:
        json.dump(obj, f)
        f.write('\n')

In [65]:
# Follow instructions here to first install the firectil CLI - https://readme.fireworks.ai/docs/fine-tuning-models#installing-firectl
# Then run this command to upload the file to Fireworks
!firectl create dataset {dataset_id} {dataset_file_name}

### Fine-Tuning

We will now fine-tune models using the Fireworks API. Fireworks implements the QLoRA algorithm through a simple interface. Training parameters can be set via the --settings-file argument. In this exercise, we will fine-tune two models:
- The first model will use Fireworks default training parameters
- The second model will increase the rank, learning rate, and epochs to make fine-tuning more aggressive

See https://docs.fireworks.ai/fine-tuning/fine-tuning-models#starting-your-tuning-job for more details

In [64]:
# Creates a training job with the default hyperparameters
!firectl create fine-tuning-job --settings-file ticket_classification_v1.yaml --display-name ticket-classification-v1 --dataset {dataset_id} 

In [63]:
# Creates a training job with the increased rank, learning rate, and epochs
!firectl create fine-tuning-job --settings-file ticket_classification_v2.yaml --display-name ticket-classification-v2 --dataset {dataset_id} 

In [20]:
# v1 is the id of the training job with default hyperparameters, v2 is with the increased settings
# NOTE THAT THESE IDS WILL CHANGE WHEN YOU RUN THE FINE-TUNING JOB ON YOUR ACCOUNT!!!
# The model id is printed in the stdout of the cell above as Name: accounts/{account_id}/fineTuningJobs/{model_id}
model_v1_id = 'e39cc15d4b80486ea089786b565cf7d1'
model_v2_id = 'a2b729a82fc64095a3c487b392f5187b'

In [47]:
# Wait until the State of the two fine-tuning jobs are listed as COMPLETED (~10-20 minutes)
!firectl get fine-tuning-job {model_v1_id}

### Evluate Results

We will now deploy our models and evaluate the results. We will calculate the accuracy on three different models

- The base model without any fine-tuning
- Our first fine-tuned model, with the default hyperparameters
- Our second fine-tuned model, with the more aggressive hyperparameters

See https://docs.fireworks.ai/fine-tuning/fine-tuning-models#deploying-the-model-for-inference for more details

In [62]:
# Deploy the first model to a Fireworks serverless endpoint
!firectl deploy {model_v1_id}

In [61]:
# Deploy the second model to a Fireworks serverless endpoint
!firectl deploy {model_v2_id}

In [60]:
# Wait until the the Deploymed Model Refs lists the state of the models as "DEPLOYED" (~5-20 minutes).
!firectl get model {model_v1_id}

In [33]:
# Uses an LLM to predicted class labels for a list of support tickets
def classify_tickets(tickets, model):
    responses = list()

    for ticket in tickets:
        user_prompt = create_prompt(ticket)
    
        response = client.chat.completions.create(
            model=model,
            messages=[
                { "role": "user", "content": user_prompt}
            ],
            # setting temperature to 0 for this use case, so that responses are as deterministic as possible
            temperature=0, 
            stop=["</category>"],
            max_tokens=2048,
        )
        response = response.choices[0].message.content.split("<category>")[-1].strip()
        responses.append(response)

    return responses


# Calculates the percent of predictions we classified correctly
def evaluate_accuracy(predicted, actual):
    num_correct = sum([predicted[i] == actual[i] for i in range(len(actual))])
    return round(100 * num_correct / len(actual), 2)

In [53]:
# Determine how the base model without any fine-tuning performs
model_id = 'accounts/fireworks/models/llama-v3-8b-instruct'

training_responses = classify_tickets(
    tickets=training_tickets, 
    model=model_id
)
accuracy = evaluate_accuracy(training_responses, training_labels)
print(f"Training Set Accuracy: {accuracy}%")

test_responses = classify_tickets(
    tickets=test_tickets, 
    model=model_id
)

accuracy = evaluate_accuracy(test_responses, test_labels)
print(f"Test Set Accuracy: {accuracy}%")

Training Set Accuracy: 60.29%
Test Set Accuracy: 60.29%


In [54]:
# Determine how the fine-tuned model performs with the default fine-tuning params
model_id = f'accounts/{account_id}/models/{model_v1_id}'

training_responses = classify_tickets(
    tickets=training_tickets, 
    model=model_id
)
accuracy = evaluate_accuracy(training_responses, training_labels)
print(f"Training Set Accuracy: {accuracy}%")

test_responses = classify_tickets(
    tickets=test_tickets, 
    model=model_id
)

accuracy = evaluate_accuracy(test_responses, test_labels)
print(f"Test Set Accuracy: {accuracy}%")

Training Set Accuracy: 69.12%
Test Set Accuracy: 67.65%


In [55]:
# Determine how the base model performs with the increases rank, epochs, and learning rate
model_id = f'accounts/{account_id}/models/{model_v2_id}'

training_responses = classify_tickets(
    tickets=training_tickets, 
    model=model_id
)
accuracy = evaluate_accuracy(training_responses, training_labels)
print(f"Training Set Accuracy: {accuracy}%")

test_responses = classify_tickets(
    tickets=test_tickets, 
    model=model_id
)

accuracy = evaluate_accuracy(test_responses, test_labels)
print(f"Test Set Accuracy: {accuracy}%")

Training Set Accuracy: 61.76%
Test Set Accuracy: 55.88%


In [58]:
# Undeploy the first model (does not cost anything extra, but Fireworks may limit your number of deployed models).
!firectl undeploy {model_v1_id}

In [59]:
# Undeploy the second model (does not cost anything extra, but Fireworks may limit your number of deployed models).
!firectl undeploy {model_v2_id}